In [5]:
%run ../common_utils.py

# Import Data

In [6]:
train, test, metadata = load_all_data()

In [7]:
nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
categorical.remove('district')
all_features = list(train.columns)
all_features.remove('price')
numerical_features = ['area_total','area_kitchen','area_living','floor','rooms','ceiling',
    'bathrooms_shared','bathrooms_private','balconies','loggias','phones','building_id','constructed','stories']

In [9]:
categorical_to_numerical(train, ['street','address'])
categorical_to_numerical(test, ['street','address'])

In [11]:
%%capture --no-display
X_train, y_train, X_test, y_test, test_labels = pre_process_numerical(features = all_features, Numerical_features = numerical_features, train = train, test = test,
                    outliers_value=7, val_split=0.2, random_state=42, scaler="std",
                    add_R="True", add_rel_height="True", droptable=[],
                    one_hot_encode=True, cat_features=categorical, drop_old=True)

# Load Models

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from scipy import stats
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb


In [31]:
catboost = CatBoostRegressor(
    loss_function='MSLE',
    n_estimators=500,
    learning_rate=0.1,
    thread_count=-1,
    depth=7,
    silent=True,
    random_seed=42,
    bagging_temperature=0.2
#     od_type="Iter"
)

In [16]:
light_gbm = lgb.LGBMRegressor(
    num_leaves=10,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=2000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

In [17]:
grad_boost_regr = GradientBoostingRegressor(
    learning_rate=0.01,
    n_estimators=2000,
    subsample=1.0, 
    criterion='mse', 
    min_samples_split=4, 
    min_samples_leaf=2, 
    min_weight_fraction_leaf=0.0, 
    max_depth=9, 
    min_impurity_decrease=0.0, 
    init=None, 
    random_state=0, 
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0
)

In [18]:
adaboost = AdaBoostRegressor(
    n_estimators=1500,
    learning_rate=0.05,
    loss='exponential',
    random_state=42
)

In [21]:
xgboost = xgb.XGBRegressor(
    eta = 0.1, 
    max_depth=10, 
    min_child_weight=1,  
    eval_metric = 'rmsle', 
    n_estimators=500,
    n_jobs=-1 , 
    verbose=1,
    learning_rate=0.16, 
    seed=42
)

# Metrics

In [22]:
from sklearn import metrics

In [23]:
metrics.mean_squared_log_error

<function sklearn.metrics._regression.mean_squared_log_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average')>

In [32]:
catboost.fit(X_train, y_train)

CatBoostError: Invalid loss_function='MSLE': for regressor use RMSE, MultiRMSE, SurvivalAft, MAE, Quantile, LogLinQuantile, Poisson, MAPE, Lq or custom objective object